In [1]:
%pip install qiskit_aer
%pip install qiskit
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

backend = StatevectorSampler()

# bolchSphereCalc
## A class to calculate the Bloch sphere coordinates (X, Y, Z) for a given quantum state.
### Attributes:
- beta (float): The beta parameter for initializing the quantum state.
- teta (float): The teta parameter for initializing the quantum state.
- shots (int): The number of shots for the quantum circuit simulation. Default is 8000.
- X (float): The X coordinate on the Bloch sphere.
- Y (float): The Y coordinate on the Bloch sphere.
- Z (float): The Z coordinate on the Bloch sphere.
- qc (QuantumCircuit): The quantum circuit used for the calculations.
### Methods:
- initCircuit():
```
    Initializes the quantum circuit with the given beta and teta parameters.
```
- getProba(c):
```
    Calculates the probability difference between measurement outcomes '0' and '1'.
```
- initX():
```
    Initializes the X coordinate on the Bloch sphere by applying a Hadamard gate and measuring the circuit.
```
- initY():
```
    Initializes the Y coordinate on the Bloch sphere by applying an S-dagger gate, a Hadamard gate, and measuring the circuit.
```
- initZ():
```
    Initializes the Z coordinate on the Bloch sphere by measuring the circuit directly.
```
- initAll():
```
    Initializes all three coordinates (X, Y, Z) on the Bloch sphere.
```

In [3]:
class bolchSphereCalc:
    def __init__(self, beta, teta, shots=8000):
        self.X = 0
        self.Y = 0
        self.Z = 0
        self.beta = beta
        self.teta = teta
        self.shots = shots
        self.qc = QuantumCircuit(1)
        self.initCircuit()

    def initCircuit(self):
        self.qc.initialize([self.beta, self.teta], 0)

    def getProba(self, c):
        p0 = c.get('0', 0) / self.shots
        p1 = c.get('1', 0) / self.shots
        return (p0 - p1)

    def initX(self):
        tmpCirc = self.qc.copy()
        tmpCirc.h(0)
        tmpCirc.measure_all()
        print("xCircuit:")
        print(tmpCirc.draw())
        res = backend.run([tmpCirc], shots=self.shots).result()[0] # ne pas utiliser aersim / use statevectorsample
        print(res.data)
        c = res.data.meas.get_counts()
        self.X = self.getProba(c)

    def initY(self):
        tmpCirc = self.qc.copy()
        tmpCirc.sdg(0)
        tmpCirc.h(0)
        tmpCirc.measure_all()
        print("yCircuit:")
        print(tmpCirc.draw())
        res = backend.run([tmpCirc], shots=self.shots).result()[0]
        print(res)
        c = res.data.meas.get_counts()
        self.Y = self.getProba(c)

    def initZ(self):
        tmpCirc = self.qc.copy()
        tmpCirc.measure_all()
        print("zCircuit:")
        print(tmpCirc.draw())
        res = backend.run([tmpCirc], shots=self.shots).result()[0]
        print(res)
        c = res.data.meas.get_counts()
        self.Z = self.getProba(c)

    def initAll(self):
        self.initZ()
        self.initX()
        self.initY()

In [4]:
import numpy as np

# test
## A test program to check if bloch coordinates are good

- Args:
    - res: An object with attributes `X`, `Y`, and `Z` representing the result values.
    - expect: A list or tuple containing the expected values in the order [X, Z, Y].

In [5]:
def test(beta, teta):
    exCour = bolchSphereCalc(beta, teta)
    exCour.initAll()
    return (exCour)

In [6]:
res = test(np.sqrt(2/3), np.sqrt(1/3))
expect = [2 * np.sqrt(2) / 3, 0, 1 / 3]
print(f"expected: [x: {expect[0]}, z: {expect[2]}, y: {expect[1]}]\nobtain: [x: {res.X}, z: {res.Z}, y: {res.Y}]")

zCircuit:
        ┌────────────────────────────┐ ░ ┌─┐
     q: ┤ Initialize(0.8165,0.57735) ├─░─┤M├
        └────────────────────────────┘ ░ └╥┘
meas: 1/══════════════════════════════════╩═
                                          0 
SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=8000, num_bits=1>)), metadata={'shots': 8000, 'circuit_metadata': {}})
xCircuit:
        ┌────────────────────────────┐┌───┐ ░ ┌─┐
     q: ┤ Initialize(0.8165,0.57735) ├┤ H ├─░─┤M├
        └────────────────────────────┘└───┘ ░ └╥┘
meas: 1/═══════════════════════════════════════╩═
                                               0 
DataBin(meas=BitArray(<shape=(), num_shots=8000, num_bits=1>))
yCircuit:
        ┌────────────────────────────┐┌─────┐┌───┐ ░ ┌─┐
     q: ┤ Initialize(0.8165,0.57735) ├┤ Sdg ├┤ H ├─░─┤M├
        └────────────────────────────┘└─────┘└───┘ ░ └╥┘
meas: 1/══════════════════════════════════════════════╩═
                                                      0 
SamplerPubR